In [1]:
from importlib import reload
#reload(Utilities)
#reload(clm)

import sys, os
import re
import pickle

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version

import copy
import itertools
import adjustText

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
from AbsPerfectPower import AbsPerfectPower as appwr 
#-----
from DOVSAudit import DOVSAudit
from DABatch import DABatch
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Utilities_dt
from Utilities_df import DFConstructType
import Plot_General
import Plot_Box_sns
import Plot_Hist
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSingleSlicer as DFSingleSlicer

In [2]:
def delta_end_build_helper(
    df_i         , 
    return_abs   = True, 
    end_time_col = 'end_time', 
    DT_ON_TS_col = 'DT_ON_TS'
):
    r"""
    """
    #-------------------------
    return_list = [
        x - df_i[end_time_col] 
        for x in df_i[DT_ON_TS_col]
    ]
    if return_abs:
        return_list = [np.abs(x) for x in return_list]
    return return_list

def delta_end_comp_helper(
    df_i              , 
    max_delta         , 
    how               = 'any', 
    abs_delta_end_col = 'abs_delta_end', 
):
    r"""
    """
    #-------------------------
    bool_list = [x < max_delta for x in df_i[abs_delta_end_col]]
    if how == 'any':
        return np.any(bool_list)
    elif how == 'all':
        return np.all(bool_list)
    else:
        assert(0)

In [4]:
def merge_and_match_alarms_and_dovs(
    alarms_df            , 
    dovs_df              , 
    is_dovs_consolidated , 
    max_match_delta_dt_beg ,
    max_match_delta_dt_end = None,
    #-----
    how                    = 'inner', 
    left_on                = 'location_nb', 
    right_on               = 'LOCATION_ID', 
    #-----
    t_beg_col_alrm         = 'start_time', 
    t_end_col_alrm         = 'end_time', 
    #-----
    t_beg_col_dovs         = 'DT_OFF_TS_FULL', 
    t_end_col_dovs         = 'DT_ON_TS', 
):
    r"""
    Returns merged_df, matched_df
    """
    #--------------------------------------------------
    if max_match_delta_dt_end is None:
        max_match_delta_dt_end = max_match_delta_dt_beg
    #--------------------------------------------------
    merged_df = pd.merge(
        alarms_df, 
        dovs_df, 
        how      = how, 
        left_on  = left_on, 
        right_on = right_on
    )
    #--------------------------------------------------
    merged_df['delta_start']     = merged_df[t_beg_col_dovs] - merged_df[t_beg_col_alrm]
    merged_df['abs_delta_start'] = np.abs(merged_df['delta_start'])
    if is_dovs_consolidated:
        merged_df['delta_end'] = merged_df.apply(
            lambda x: delta_end_build_helper(
                df_i         = x, 
                return_abs   = False, 
                end_time_col = t_end_col_alrm, 
                DT_ON_TS_col = t_end_col_dovs
            ), 
            axis=1
        )
        merged_df['abs_delta_end'] = merged_df.apply(
            lambda x: delta_end_build_helper(
                df_i         = x, 
                return_abs   = True, 
                end_time_col = t_end_col_alrm, 
                DT_ON_TS_col = t_end_col_dovs
            ), 
            axis=1
        )
        #-----
        matched_df = merged_df[
            (merged_df['abs_delta_start']<max_match_delta_dt_beg) & 
            (
                merged_df.apply(
                    lambda x: delta_end_comp_helper(
                        df_i              = x, 
                        max_delta         = max_match_delta_dt_end, 
                        how               = 'any', 
                        abs_delta_end_col = 'abs_delta_end', 
                    ), 
                    axis=1
                )
            )
        ]
    else:
        merged_df['delta_end']     = merged_df[t_end_col_dovs] - merged_df[t_end_col_alrm]
        merged_df['abs_delta_end'] = np.abs(merged_df['delta_end'])
        #-----
        matched_df = merged_df[
            (merged_df['abs_delta_start'] < max_match_delta_dt_beg) & 
            (merged_df['abs_delta_end']   < max_match_delta_dt_end)
        ]
    #--------------------------------------------------
    return merged_df, matched_df

In [7]:
# max_match_delta_dt_beg = pd.Timedelta('2hour')
# max_match_delta_dt_end = pd.Timedelta('1200hours')

max_match_delta_dt_beg = pd.Timedelta('1day')
max_match_delta_dt_end = pd.Timedelta('1day')
#-----
long_alarm_len         = pd.Timedelta('5min')
consolidate_dovs       = False

In [9]:
alarms_df = pd.read_csv(r'C:\Users\s346557\Downloads\alarms_ohio.csv')
alarms_df.head()

,circuit_name,equipment_name,location_nb,tag,attribute_name,value,start_time,end_time,duration_seconds
0,Station Equipment,CB K,NaN,00036_TX1_CBK_FLTTRIP_A.Scan Value,Fault Trip - Phase A,ALARM,2023-11-08 15:46:06,2023-11-08 15:49:43,217.0
1,Station Equipment,CB K,NaN,00036_TX1_CBK_FLTTRIP_B.Scan Value,Fault Trip - Phase B,ALARM,2023-11-08 15:46:33,2023-11-08 15:49:43,190.0
2,Station Equipment,CB K,NaN,00036_TX1_CBK_FLTTRIP_C.Scan Value,Fault Trip - Phase C,ALARM,2023-11-08 15:47:05,2023-11-08 15:49:43,158.0
3,Station Equipment,CB L,NaN,00036_TX2_CBL_FLTTRIP_A.Scan Value,Fault Trip - Phase A,ALARM,2023-11-08 16:06:20,2023-11-08 16:09:42,202.0
4,Station Equipment,CB L,NaN,00036_TX2_CBL_FLTTRIP_B.Scan Value,Fault Trip - Phase B,ALARM,2023-11-08 16:06:47,2023-11-08 16:09:42,175.0


In [16]:
#-------------------------
# Some entries (34) have end_time = 2154-12-31 00:00:00
# Not sure what these are, so remove
alarms_df = alarms_df[alarms_df['end_time']!=alarms_df['end_time'].max()].copy()

#-------------------------
# Remove any with NaN location_nb
alarms_df = alarms_df.dropna(subset=['location_nb']).copy()

#-------------------------
# Data type conversions
alarms_df['location_nb'] = alarms_df['location_nb'].astype(np.int64).astype(str)
#-----
alarms_df = Utilities_df.convert_col_types(
    df = alarms_df, 
    cols_and_types_dict = {k:datetime.datetime for k in ['start_time', 'end_time']},
    to_numeric_errors='coerce',
    inplace=False,    
)
#-----
alarms_df['duration'] = pd.to_timedelta(alarms_df['duration_seconds'], unit='seconds')
#-----
alarms_df['circuit_nb'] = alarms_df['circuit_name'].str.extract(r'.*\((\d*)\)')

In [19]:
# For now, to eliminate any confusion, only consider:
#     'location_nb', 'start_time', 'end_time', 'duration_seconds'
#   and drop any duplicates
alarms_df[['location_nb', 'start_time', 'end_time', 'duration_seconds']].value_counts()

location_nb    start_time           end_time             duration_seconds
40820529       2024-04-12 17:31:41  2024-04-12 17:31:44  3.0                 5
41810746       2024-08-01 16:12:28  2024-08-01 16:12:31  3.0                 4
1866120727926  2024-02-28 14:13:29  2024-02-28 14:13:35  6.0                 4
1878494582764  2024-07-25 06:57:02  2024-07-25 06:57:08  6.0                 4
1923546768926  2024-05-24 19:44:43  2024-05-24 19:44:44  1.0                 4
                                                                            ..
1852922721331  2024-08-26 10:52:36  2024-08-26 10:52:52  16.0                1
2092949299815  2024-08-16 11:17:41  2024-08-16 11:17:45  4.0                 1
1852922721331  2024-08-26 10:52:35  2024-08-26 10:52:52  17.0                1
1852200800177  2024-06-23 19:12:37  2024-06-23 19:12:40  3.0                 1
1678001440756  2024-06-24 07:36:41  2024-06-24 07:36:41  0.0                 1
Name: count, Length: 3211, dtype: int64

In [21]:
alarms_df[(alarms_df['location_nb']=='40820529') & (alarms_df['start_time']=='2024-04-12 17:31:41')]

,circuit_name,equipment_name,location_nb,tag,attribute_name,value,start_time,end_time,duration_seconds,duration,circuit_nb
4930,WEST NEWARK (7401403),Recloser - 40820529C10057,40820529,3000533864_FLTTRIP_B.Scan Value,Fault Trip - Phase B,ALARM,2024-04-12 17:31:41,2024-04-12 17:31:44,3.0,0 days 00:00:03,7401403
4931,WEST NEWARK (7401403),Recloser - 40820529C10057,40820529,3000533864_FLTTRIP_C.Scan Value,Fault Trip - Phase C,ALARM,2024-04-12 17:31:41,2024-04-12 17:31:44,3.0,0 days 00:00:03,7401403
4933,WEST NEWARK (7401403),Recloser - 40820529C10057,40820529,3000533864_FLTTRIP_G.Scan Value,Fault Trip - Ground,ALARM,2024-04-12 17:31:41,2024-04-12 17:31:44,3.0,0 days 00:00:03,7401403
4963,WEST NEWARK (7401403),Recloser - 40820529C10598,40820529,3000534715_FLTTRIP_B.Scan Value,Fault Trip - Phase B,ALARM,2024-04-12 17:31:41,2024-04-12 17:31:44,3.0,0 days 00:00:03,7401403
4966,WEST NEWARK (7401403),Recloser - 40820529C10598,40820529,3000534715_FLTTRIP_G.Scan Value,Fault Trip - Ground,ALARM,2024-04-12 17:31:41,2024-04-12 17:31:44,3.0,0 days 00:00:03,7401403


In [24]:
# For now, to eliminate any confusion, only consider:
#     'location_nb', 'start_time', 'end_time', 'duration_seconds'
#   and drop any duplicates
alarms_df_full = alarms_df.copy()
alarms_df = alarms_df[['location_nb', 'start_time', 'end_time', 'duration_seconds', 'duration']].drop_duplicates()
#-----
long_alarms_df = alarms_df[alarms_df['duration']>long_alarm_len].copy()

In [26]:
alarms_df[['location_nb', 'start_time', 'end_time', 'duration_seconds']].value_counts()

location_nb    start_time           end_time             duration_seconds
1678001440756  2024-06-24 07:36:41  2024-06-24 07:36:41  0.0                 1
2082458445161  2024-05-23 10:28:48  2024-05-23 10:28:51  3.0                 1
2081272279077  2024-06-28 07:56:13  2024-06-28 07:56:16  3.0                 1
               2024-07-03 12:13:28  2024-07-03 12:13:31  3.0                 1
               2024-07-03 15:52:13  2024-07-03 15:52:16  3.0                 1
                                                                            ..
1862229726342  2024-06-22 19:43:30  2024-06-23 05:18:01  34471.0             1
1862693410627  2024-05-08 00:16:01  2024-05-08 00:16:04  3.0                 1
               2024-05-08 00:16:05  2024-05-08 00:16:10  5.0                 1
               2024-05-08 00:16:06  2024-05-08 00:16:10  4.0                 1
41841006       2024-10-17 13:36:04  2024-10-17 13:36:08  4.0                 1
Name: count, Length: 3211, dtype: int64

In [28]:
alarms_df[['location_nb', 'start_time']].value_counts()

location_nb    start_time         
41811053       2024-04-23 14:24:00    4
1923546768926  2024-05-24 19:44:46    3
40830528       2024-10-03 00:39:26    3
1898715729829  2024-02-28 05:46:10    2
1873023759844  2024-07-23 07:55:13    2
                                     ..
1862891741244  2024-07-15 17:03:55    1
1862892444981  2024-09-27 14:46:47    1
               2024-09-27 14:46:52    1
               2024-09-27 14:46:58    1
41841006       2024-10-17 13:36:04    1
Name: count, Length: 3098, dtype: int64

In [30]:
alarms_df[(alarms_df['location_nb']=='41811053') & (alarms_df['start_time']=='2024-04-23 14:24:00')]

,location_nb,start_time,end_time,duration_seconds,duration
4559,41811053,2024-04-23 14:24:00,2024-04-23 14:24:03,3.0,0 days 00:00:03
4560,41811053,2024-04-23 14:24:00,2024-04-23 14:24:01,1.0,0 days 00:00:01
4563,41811053,2024-04-23 14:24:00,2024-04-23 14:24:00,0.0,0 days 00:00:00
4585,41811053,2024-04-23 14:24:00,2024-04-23 16:48:00,8640.0,0 days 02:24:00


In [34]:
# _v2 utilizes a consolidation technique
alarms_df_v2 = appwr.combine_zero_times_by_PN(
    zero_times_df = alarms_df, 
    t_min_col     = 'start_time', 
    t_max_col     = 'end_time', 
    PN_col        = 'location_nb',    
)
alarms_df_v2['duration'] = alarms_df_v2['end_time'] - alarms_df_v2['start_time']
alarms_df_v2

,location_nb,start_time,end_time,duration
0,1678001440756,2024-06-24 07:36:41,2024-06-24 07:36:44,0 days 00:00:03
1,1678001440756,2024-06-26 18:56:20,2024-06-26 18:56:23,0 days 00:00:03
2,1678001440756,2024-07-06 17:19:40,2024-07-06 17:19:43,0 days 00:00:03
3,1678001440756,2024-09-27 06:01:15,2024-09-27 06:01:18,0 days 00:00:03
4,1678001440756,2024-09-27 06:01:30,2024-09-27 06:01:35,0 days 00:00:05
...,...,...,...,...
2853,41841006,2024-03-02 12:30:55,2024-03-02 12:30:59,0 days 00:00:04
2854,41841006,2024-03-11 10:16:47,2024-03-11 10:16:50,0 days 00:00:03
2855,41841006,2024-03-14 05:52:58,2024-03-14 08:35:01,0 days 02:42:03
2856,41841006,2024-03-28 12:13:09,2024-03-28 12:13:18,0 days 00:00:09


In [39]:
# Dummy DOVS to output sample SQL
build_sql_function_kwargs = dict(
    location_ids     = ['ID_1', 'ID_2', 'ID_n'], 
    date_range       = [alarms_df['start_time'].min(), alarms_df['end_time'].max()], 
    field_to_split   = 'location_ids', 
    batch_size       = 1000
)
#-----
dovs = DOVSOutages(
    df_construct_type         = DFConstructType.kRunSqlQuery, 
    contstruct_df_args        = None, 
    init_df_in_constructor    = False, 
    build_sql_function        = DOVSOutages_SQL.build_sql_outage, 
    build_sql_function_kwargs = build_sql_function_kwargs, 
    save_args                 = False, 
    build_consolidated        = False
)
dovs.get_sql().print()

SELECT
	DOV.CI_NB,
	DOV.CMI_NB,
	DOV.OUTG_REC_NB,
	DOV.OUTAGE_NB,
	DOV.DT_ON_TS,
	DOV.DT_OFF_TS,
	DOV.DT_ON_TS - DOV.STEP_DRTN_NB/(60*24) AS DT_OFF_TS_FULL,
	DOV.STEP_DRTN_NB,
	EXTRACT(YEAR FROM DOV.DT_OFF_TS) AS START_YEAR,
	DOV.OPERATING_UNIT_ID,
	DOV.STATE_ABBR_TX,
	DOV.MJR_CAUSE_CD,
	DOV.MNR_CAUSE_CD,
	DOV.LOCATION_ID,
	DOV.GIS_CRCT_NB
FROM DOVSADM.DOVS_OUTAGE_FACT DOV
WHERE DOV.DT_OFF_TS BETWEEN '2023-11-01' AND '2024-11-14'
AND   DOV.LOCATION_ID IN ('ID_1','ID_2','ID_n')


In [41]:
# build_sql_function_kwargs = dict(
#     mjr_mnr_cause    = None, 
#     include_premise  = False,
#     location_ids     = alarms_df['location_nb'].unique().tolist(), 
#     date_range       = [alarms_df['start_time'].min(), alarms_df['end_time'].max()], 
#     field_to_split   = 'location_ids', 
#     MJR_CAUSE_CD     = None, 
#     DEVICE_CD        = None, 
#     INTRPTN_TYP_CD   = None, 
#     CURR_REC_STAT_CD = None, 
#     batch_size       = 1000
# )
# #-----
# dovs = DOVSOutages(
#     df_construct_type         = DFConstructType.kRunSqlQuery, 
#     contstruct_df_args        = None, 
#     init_df_in_constructor    = True, 
#     build_sql_function        = DOVSOutages_SQL.build_sql_std_outage, 
#     build_sql_function_kwargs = build_sql_function_kwargs, 
#     save_args                 = False, 
#     build_consolidated        = False
# )
# dovs_df = dovs.df

In [43]:
build_sql_function_kwargs = dict(
    location_ids     = alarms_df['location_nb'].unique().tolist(), 
    date_range       = [alarms_df['start_time'].min(), alarms_df['end_time'].max()], 
    field_to_split   = 'location_ids', 
    batch_size       = 1000
)
#-----
dovs = DOVSOutages(
    df_construct_type         = DFConstructType.kRunSqlQuery, 
    contstruct_df_args        = None, 
    init_df_in_constructor    = True, 
    build_sql_function        = DOVSOutages_SQL.build_sql_outage, 
    build_sql_function_kwargs = build_sql_function_kwargs, 
    save_args                 = False, 
    build_consolidated        = False
)
dovs_df = dovs.df

n_coll     = 504
batch_size = 1000
n_batches  = 1


C:\Users\s346557\Documents\Analysis\GenAn.py:663: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn_db, **read_sql_args)


In [45]:
dovs_df.head()

,CI_NB,CMI_NB,OUTG_REC_NB,OUTAGE_NB,DT_ON_TS,DT_OFF_TS,DT_OFF_TS_FULL,STEP_DRTN_NB,START_YEAR,OPERATING_UNIT_ID,STATE_ABBR_TX,MJR_CAUSE_CD,MNR_CAUSE_CD,LOCATION_ID,GIS_CRCT_NB
0,14,15430.0,13815187,3189671,2024-03-15 15:35:00,2024-03-14,2024-03-14 21:08:00,1107.0,2024,3,OH,DL,WT,1884864818392,0027032
1,2451,2582333.0,13815189,3189671,2024-03-15 15:35:00,2024-03-14,2024-03-14 21:08:00,1107.0,2024,3,OH,DL,WT,1884864818392,0027032
2,0,0.0,13815190,3189671,2024-03-15 15:35:00,2024-03-14,2024-03-14 21:10:00,1105.0,2024,3,OH,NI,DOT,1884864818392,0009231
3,2943,2806920.0,13815201,3189381,2024-03-16 13:01:00,2024-03-14,2024-03-14 21:06:00,2395.0,2024,3,OH,DL,WT,1846554817073,0024033
4,0,0.0,14077389,3871151,2024-08-08 09:14:00,2024-08-08,2024-08-08 09:03:00,11.0,2024,3,OH,NI,AE,1930977725255,7368798


In [46]:
# A single LOCATION_ID can be responsible for multiple outages
# For simplicity, if desired, consolidate dovs_df
dovs_cols_to_keep = [
    'CI_NB',
    'CMI_NB',
    'OUTG_REC_NB',
    'DT_ON_TS',
    'DT_OFF_TS_FULL',
    'MJR_CAUSE_CD',
    'MNR_CAUSE_CD',
    'LOCATION_ID'
]
#-----
dovs_df[['LOCATION_ID', 'DT_OFF_TS_FULL']].value_counts()

LOCATION_ID     DT_OFF_TS_FULL     
2381317530613   2024-07-10 12:14:00    5
1812346308218   2024-10-02 07:12:00    4
1898715729829   2024-07-12 16:18:00    4
1890047762896   2024-11-13 06:38:00    3
1889001757939   2024-08-16 21:15:00    3
                                      ..
1866061753272   2024-06-28 20:06:00    1
                2024-11-10 11:45:00    1
1866120727926   2024-02-28 14:24:00    1
                2024-03-31 18:01:00    1
41821032000394  2024-07-26 17:20:00    1
Name: count, Length: 214, dtype: int64

In [47]:
if consolidate_dovs:
    dovs_df = Utilities_df.consolidate_df(
        df                                  = dovs_df[dovs_cols_to_keep].copy(), 
        groupby_cols                        = ['LOCATION_ID', 'DT_OFF_TS_FULL'], 
        cols_shared_by_group                = None, 
        cols_to_collect_in_lists            = None, 
        as_index                            = False, 
        include_groupby_cols_in_output_cols = False, 
        allow_duplicates_in_lists           = False, 
        allow_NaNs_in_lists                 = False, 
        recover_uniqueness_violators        = True, 
        gpby_dropna                         = True, 
        rename_cols                         = None, 
        custom_aggs_for_list_cols           = None, 
        verbose                             = True
    )
else:
    dovs_df = dovs_df[dovs_cols_to_keep].copy()

In [48]:
dovs_df.head()

,CI_NB,CMI_NB,OUTG_REC_NB,DT_ON_TS,DT_OFF_TS_FULL,MJR_CAUSE_CD,MNR_CAUSE_CD,LOCATION_ID
0,14,15430.0,13815187,2024-03-15 15:35:00,2024-03-14 21:08:00,DL,WT,1884864818392
1,2451,2582333.0,13815189,2024-03-15 15:35:00,2024-03-14 21:08:00,DL,WT,1884864818392
2,0,0.0,13815190,2024-03-15 15:35:00,2024-03-14 21:10:00,NI,DOT,1884864818392
3,2943,2806920.0,13815201,2024-03-16 13:01:00,2024-03-14 21:06:00,DL,WT,1846554817073
4,0,0.0,14077389,2024-08-08 09:14:00,2024-08-08 09:03:00,NI,AE,1930977725255


# Deltas convention
***
delta = DOVS - ALARM

- delta_start:
    - delta_start > 0 ==> DOVS event began AFTER  alarm began
    - delta_start < 0 ==> DOVS event began BEFORE alarm began
 
- delta_end:
    - delta_end > 0 ==> DOVS event ended AFTER  alarm ended
    - delta_end < 0 ==> DOVS event ended BEFORE alarm ended

In [51]:
merged_df, matched_df = merge_and_match_alarms_and_dovs(
    alarms_df              = alarms_df, 
    dovs_df                = dovs_df, 
    is_dovs_consolidated   = consolidate_dovs, 
    max_match_delta_dt_beg = max_match_delta_dt_beg, 
    max_match_delta_dt_end = max_match_delta_dt_end, 
    #-----
    how                    = 'inner', 
    left_on                = 'location_nb', 
    right_on               = 'LOCATION_ID', 
)

In [52]:
merged_df_v2, matched_df_v2 = merge_and_match_alarms_and_dovs(
    alarms_df              = alarms_df_v2, 
    dovs_df                = dovs_df, 
    is_dovs_consolidated   = consolidate_dovs, 
    max_match_delta_dt_beg = max_match_delta_dt_beg, 
    max_match_delta_dt_end = max_match_delta_dt_end, 
    #-----
    how                    = 'inner', 
    left_on                = 'location_nb', 
    right_on               = 'LOCATION_ID', 
)

In [53]:
matched_df.shape

(666, 17)

In [54]:
matched_df[['location_nb', 'start_time', 'end_time']].value_counts().shape[0]

465

In [60]:
vcs             = matched_df[['location_nb', 'start_time', 'end_time']].value_counts()
mult_matched_df = matched_df.set_index(['location_nb', 'start_time', 'end_time']).loc[vcs[vcs>1].index].copy()
mult_matched_df

duration_seconds  \
location_nb   start_time          end_time                                
2381317530613 2024-07-10 12:13:47 2024-07-10 12:48:51            2104.0   
                                  2024-07-10 12:48:51            2104.0   
                                  2024-07-10 12:48:51            2104.0   
                                  2024-07-10 12:48:51            2104.0   
                                  2024-07-10 12:48:51            2104.0   
...                                                                 ...   
1912968735595 2024-02-28 05:49:52 2024-02-28 05:49:54               2.0   
              2024-02-28 05:50:14 2024-02-28 05:50:15               1.0   
                                  2024-02-28 05:50:15               1.0   
1901147746602 2023-11-26 18:56:31 2023-11-26 18:56:34               3.0   
                                  2023-11-26 18:56:34               3.0   

                                                             duration  CI_NB  \
location_nb   start_time          end_time                                     
2381317530613 2024-07-10 12:13:47 2024-07-10 12:48:51 0 days 00:35:04    136   
                                  2024-07-10 12:48:51 0 days 00:35:04      0   
                                  2024-07-10 12:48:51 0 days 00:35:04    563   
                                  2024-07-10 12:48:51 0 days 00:35:04     73   
                                  2024-07-10 12:48:51 0 days 00:35:04    136   
...                                                               ...    ...   
1912968735595 2024-02-28 05:49:52 2024-02-28 05:49:54 0 days 00:00:02      0   
              2024-02-28 05:50:14 2024-02-28 05:50:15 0 days 00:00:01      0   
                                  2024-02-28 05:50:15 0 days 00:00:01      0   
1901147746602 2023-11-26 18:56:31 2023-11-26 18:56:34 0 days 00:00:03      0   
                                  2023-11-26 18:56:34 0 days 00:00:03      0   

                                                        CMI_NB OUTG_REC_NB  \
location_nb   start_time          end_time                                   
2381317530613 2024-07-10 12:13:47 2024-07-10 12:48:51  29104.0    14028411   
                                  2024-07-10 12:48:51      0.0    14028466   
                                  2024-07-10 12:48:51  46729.0    14027949   
                                  2024-07-10 12:48:51   6059.0    14027950   
                                  2024-07-10 12:48:51  11288.0    14027955   
...                                                        ...         ...   
1912968735595 2024-02-28 05:49:52 2024-02-28 05:49:54      0.0    13795106   
              2024-02-28 05:50:14 2024-02-28 05:50:15      0.0    13795105   
                                  2024-02-28 05:50:15      0.0    13795106   
1901147746602 2023-11-26 18:56:31 2023-11-26 18:56:34      0.0    13684116   
                                  2023-11-26 18:56:34      0.0    13684215   

                                                                 DT_ON_TS  \
location_nb   start_time          end_time                                  
2381317530613 2024-07-10 12:13:47 2024-07-10 12:48:51 2024-07-10 15:48:00   
                                  2024-07-10 12:48:51 2024-07-10 13:37:00   
                                  2024-07-10 12:48:51 2024-07-10 13:37:00   
                                  2024-07-10 12:48:51 2024-07-10 13:37:00   
                                  2024-07-10 12:48:51 2024-07-10 13:37:00   
...                                                                   ...   
1912968735595 2024-02-28 05:49:52 2024-02-28 05:49:54 2024-02-29 01:31:00   
              2024-02-28 05:50:14 2024-02-28 05:50:15 2024-02-29 01:31:00   
                                  2024-02-28 05:50:15 2024-02-29 01:31:00   
1901147746602 2023-11-26 18:56:31 2023-11-26 18:56:34 2023-11-26 21:08:00   
                                  2023-11-26 18:56:34 2023-11-27 08:56:00   

                     

In [67]:
# long_matched_df
matched_long = list(
    set(long_alarms_df[['location_nb', 'start_time', 'end_time']].value_counts().index).intersection(
    set(matched_df[['location_nb', 'start_time', 'end_time']].value_counts().index)
    )
)
#-----
long_matched_df = matched_df.set_index(['location_nb', 'start_time', 'end_time']).loc[matched_long].copy()

In [70]:
unmatched_long = list(
    set(long_alarms_df[['location_nb', 'start_time', 'end_time']].value_counts().index).difference(
    set(matched_df[['location_nb', 'start_time', 'end_time']].value_counts().index)
    )
)
#-----
long_unmatched_df = alarms_df_full.set_index(['location_nb', 'start_time', 'end_time']).loc[unmatched_long].copy()

In [ ]:
print("# ('location_nb', 'start_time', 'end_time') Groups Found:")
print(f"\tmatched_df:      {matched_df[['location_nb', 'start_time', 'end_time']].value_counts().shape[0]}")
print(f"\tlong_matched_df: {long_matched_df.index.value_counts().shape[0]}")
print(f"\tlong_alarms_df:  {long_alarms_df[['location_nb', 'start_time', 'end_time']].value_counts().shape[0]}")
print(f"\talarms_df:       {alarms_df[['location_nb', 'start_time', 'end_time']].value_counts().shape[0]}")
print('\n')
print("# location_nbs Found:")
print(f"\tmatched_df:      {matched_df['location_nb'].nunique()}")
print(f"\tlong_matched_df: {long_matched_df.index.get_level_values(0).nunique()}")
print(f"\tlong_alarms_df:  {long_alarms_df['location_nb'].nunique()}")
print(f"\talarms_df:       {alarms_df['location_nb'].nunique()}")

In [ ]:
matched_df[['delta_start', 'delta_end']].describe()

In [ ]:
matched_df['delta_start_hr'] = matched_df['delta_start'].dt.total_seconds()/3600
matched_df['delta_end_hr']   = matched_df['delta_end'].dt.total_seconds()/3600

In [ ]:
x_col = 'delta_start_hr'
fig,ax = Plot_General.default_subplots()
ax = Plot_Hist.plot_hist(
    ax = ax, 
    df = matched_df, 
    x_col = x_col, 
    min_max_and_bin_size= [
        matched_df[x_col].min(), 
        matched_df[x_col].max(), 
        0.5
    ]
)
ax.set_title(x_col, fontsize='xx-large')
ax.set_xlabel('Hours', fontsize='xx-large')

In [ ]:
x_col = 'delta_start_hr'
fig,ax = Plot_General.default_subplots()
ax = Plot_Hist.plot_hist(
    ax = ax, 
    df = matched_df, 
    x_col = x_col, 
    min_max_and_bin_size= [-0.25, 0.25, 0.01], 
)
ax.set_title(x_col, fontsize='xx-large')
ax.set_xlabel('Hours', fontsize='xx-large')

In [ ]:
x_col = 'delta_end_hr'
fig,ax = Plot_General.default_subplots()
ax = Plot_Hist.plot_hist(
    ax = ax, 
    df = matched_df, 
    x_col = x_col, 
    min_max_and_bin_size= [
        matched_df[x_col].min(), 
        matched_df[x_col].max(), 
        0.5
    ]
)
ax.set_title(x_col, fontsize='xx-large')
ax.set_xlabel('Hours', fontsize='xx-large')

In [ ]:
matched_df[['delta_start', 'delta_end']].describe()

In [ ]:
matched_df.sort_values(by=['abs_delta_start'], ascending=False)

In [ ]:
matched_df['duration_seconds'].describe()

In [ ]:
alarms_df.shape

In [ ]:
alarms_df[['location_nb', 'start_time']].value_counts()

In [ ]:
alarms_df[(alarms_df['location_nb']=='41811053') & (alarms_df['start_time']=='2024-04-23 14:24:00')]

In [ ]:
matched_df[['location_nb', 'start_time']].value_counts()

In [ ]:
matched_df[['location_nb', 'start_time', 'end_time', 'duration_seconds']].value_counts()

In [ ]:
alarms_df[(alarms_df['location_nb']=='41810986000350') & (alarms_df['start_time']=='2024-01-09 08:53:15')]

In [ ]:
dovs_df[dovs_df['LOCATION_ID']=='41810986000350']

In [ ]:
matched_df_v2[['location_nb', 'start_time']].value_counts().shape

In [ ]:
alarms_df[alarms_df['duration_seconds']>60*5][['location_nb', 'start_time']].value_counts()

In [ ]:
alarms_df[(alarms_df['location_nb']=='1885717740794') & (alarms_df['start_time']=='2024-01-09 18:09:27')]

In [ ]:
alarms_df[alarms_df['duration_seconds']>60*5][['location_nb', 'start_time', 'end_time']].value_counts()

In [ ]:
alarms_df[['location_nb', 'start_time']].value_counts()

In [ ]:
alarms_df_v2[alarms_df_v2['location_nb']=='41811053']

In [ ]:
matched_df

In [ ]:
matched_df_v2[['location_nb', 'start_time']].value_counts().shape

In [ ]:
alarms_df_v2[alarms_df_v2['duration']>pd.Timedelta('5min')][['location_nb', 'start_time']].value_counts()

In [ ]:
matched_df_v2

In [ ]:
matched_df[['location_nb', 'start_time', 'end_time']].value_counts().shape[0]

In [ ]:
matched_df.shape

In [ ]:
matched_df[['location_nb', 'start_time', 'end_time']].value_counts()

In [ ]:
matched_df[(matched_df['location_nb']=='2381317530613') & (matched_df['start_time'] == '2024-07-10 12:13:47')]

In [ ]:
matched_df.set_index(['location_nb', 'start_time', 'end_time'])

In [ ]:
long_unmatched_df

In [ ]:
from collections import defaultdict, deque
import pandas as pd
import numpy as np
import oracledb

def build_tree_bilj(
    upstream, 
    node
):
    tree = defaultdict(list)
    for u, n in zip(upstream, node):
        tree[u].append(n)
    return tree


def get_downstream_ids_bilj(
    tree, 
    start_node
):
    if start_node not in tree:
        return []
    
    queue = deque([start_node])
    downstream_ids = []
    
    while queue:
        current = queue.popleft()
        downstream_ids.append(current)
        for child in tree[current]:
            queue.append(child)
    
    return downstream_ids

In [ ]:
def get_downstream_from_node_in_circuit_bilj(
    circuit_str, 
    start_node
):
    r"""
    """
    #-------------------------
    # query circuit number string
    main_circuits = f"('{circuit_str}')"
    #%%
    oracledb.init_oracle_client() 
    con_EDDSP = oracledb.connect(user = Utilities_config.get_aep_user_id(), password = Utilities.get_pwd('Oracle'), dsn = "EDDSP")
    cur = con_EDDSP.cursor()
    sql = f""" Select GIS_CIRCUIT_NB, UPSTREAM_LOCATION, LOCATION_NB
                from COMMON.COM_DGIS_CIRCUIT_DEVICES
                where GIS_CIRCUIT_NB in {main_circuits}
            """
    cur = con_EDDSP.cursor()
    cur.execute(sql)
    names = [ x[0] for x in cur.description]
    rows = cur.fetchall()
    rows = np.array(rows)
    
    EDDSP_DGIS_CIRC_DEV = pd.DataFrame(rows, columns = names)
    #-------------------------
    # Create circuit tree mapping
    tree = build_tree_bilj(EDDSP_DGIS_CIRC_DEV['UPSTREAM_LOCATION'].tolist(), EDDSP_DGIS_CIRC_DEV['LOCATION_NB'].tolist())
    #-----
    if start_node not in tree:
        print(f"WARNING: start_node={start_node} not found in circuit_str={circuit_str}!")
        print(f"circuit_str = {circuit_str}")
        print(f"start_node = {start_node}")
        print(f"tree = {list(tree.keys())}")
        print('\n')
        return None
    #-----
    node_devices = get_downstream_ids_bilj(tree, start_node)
    #-------------------------
    return node_devices

In [ ]:
#---------------------------------------------------------------------------
def get_downstream_from_nodes(
    tree, 
    nodes
):
    r"""
    Devices without anything downstream are not included in the keys of the tree
    """
    #-------------------------
    nodes_w_dwnstrm = list(set(nodes).intersection(set(tree.keys())))
    dwnstrm = list(itertools.chain.from_iterable([tree[x] for x in nodes_w_dwnstrm]))
    return dwnstrm


#---------------------------------------------------------------------------
def get_downstream_ids(
    tree       , 
    start_node , 
    max_n_itr  = 1000
):
    #--------------------------------------------------
    if start_node not in tree:
        return []
    #-------------------------
    downstream_ids = []
    n_addtnl_i     = -1
    nodes_i        = [start_node]
    #-------------------------
    n_itr = 0
    while n_addtnl_i!=0 and n_itr<max_n_itr:
        dwn_strm_lvl_i = get_downstream_from_nodes(
            tree  = tree, 
            nodes = nodes_i
        )
        addtnl_i   = list(set(dwn_strm_lvl_i).difference(set(downstream_ids)))
        n_addtnl_i = len(addtnl_i)
        if n_addtnl_i > 0:
            downstream_ids.extend(addtnl_i)
        nodes_i = dwn_strm_lvl_i
        n_itr += 1
    #-------------------------
    if n_itr==max_n_itr:
        print('WARNING: In get_downstream_from_nodes, while loop exited due to max_n_itr, not due to completion!!!!!')
    #-------------------------
    return downstream_ids

In [ ]:
def get_downstream_from_node_in_circ_dev_df_i_v0(
    circ_dev_df_i      , 
    start_node         , 
    upstrm_col         = 'UPSTREAM_LOCATION', 
    loc_nb_col         = 'LOCATION_NB', 
    include_start_node = False
):
    r"""
    """
    #-------------------------
    # Create circuit tree mapping
    tree = build_tree_bilj(
        upstream = circ_dev_df_i[upstrm_col].tolist(), 
        node     = circ_dev_df_i[loc_nb_col].tolist()
    )
    #-----
    if start_node not in tree:
        print(f"WARNING: start_node={start_node} not found!")
        print(f"start_node = {start_node}")
        print(f"tree = {list(tree.keys())}")
        print('\n')
        return None
    #-----
    node_devices = get_downstream_ids(
        tree       = tree, 
        start_node = start_node
    )
    #-------------------------
    if include_start_node:
        node_devices.append(start_node)
    #-------------------------
    return node_devices

In [ ]:
def get_downstream_from_node_in_circ_dev_df_i(
    circ_dev_df_i      , 
    start_node         , 
    upstrm_col         = 'UPSTREAM_LOCATION', 
    loc_nb_col         = 'LOCATION_NB', 
    circ_nb_col        = 'GIS_CIRCUIT_NB', 
    include_start_node = False, 
    max_n_itr          = 1000
):
    r"""
    """
    #--------------------------------------------------
    assert(circ_dev_df_i[circ_nb_col].nunique()==1)
    circ_nb_i = circ_dev_df_i[circ_nb_col].unique().tolist()[0]
    #--------------------------------------------------
    # If start_node not found in collection of upstream devices, then we cannot
    #   possibly find any downstream devices, so simply return []
    if start_node not in circ_dev_df_i[upstrm_col].unique().tolist():
        # This is already a yellow-flag, but if start_node also not found in loc_nb_col, 
        #   this becomes a red flag and the user should be notified
        if start_node not in circ_dev_df_i[loc_nb_col].unique().tolist():
            print(f'WARNING: start_node = {start_node} not found in circuit {circ_nb_i}!')
            return np.nan
        else:
            return []
    #--------------------------------------------------
    downstream_ids = []
    n_addtnl_i     = -1
    nodes_i        = [start_node]
    #-------------------------
    n_itr = 0
    while n_addtnl_i!=0 and n_itr<max_n_itr:
        nodes_i    = circ_dev_df_i[circ_dev_df_i[upstrm_col].isin(nodes_i)][loc_nb_col].unique().tolist()
        addtnl_i   = list(set(nodes_i).difference(set(downstream_ids)))
        n_addtnl_i = len(addtnl_i)
        if n_addtnl_i > 0:
            downstream_ids.extend(addtnl_i)
        n_itr += 1
    #-------------------------
    if n_itr==max_n_itr:
        print('WARNING: In get_downstream_from_node_in_circ_dev_df_i, while loop exited due to max_n_itr, not due to completion!!!!!')
    #-------------------------
    if include_start_node:
        downstream_ids.append(start_node)
    #-------------------------
    return downstream_ids


In [ ]:
def get_downstream_from_node_in_circuit_v0(
    circuit_str, 
    start_node
):
    r"""
    """
    #-------------------------
    conn_eddsp = Utilities.get_eddsp_oracle_connection()
    #-------------------------
    # query circuit number string
    main_circuits = f"('{circuit_str}')"
    sql = f""" Select GIS_CIRCUIT_NB, UPSTREAM_LOCATION, LOCATION_NB
                from COMMON.COM_DGIS_CIRCUIT_DEVICES
                where GIS_CIRCUIT_NB in {main_circuits}
            """
    circ_dev_df = pd.read_sql_query(
        sql, 
        conn_eddsp
    )
    #-------------------------
    node_devices = get_downstream_from_node_in_circ_dev_df_i_v0(
        circ_dev_df_i = circ_dev_df, 
        start_node    = start_node, 
        upstrm_col    = 'UPSTREAM_LOCATION', 
        loc_nb_col    = 'LOCATION_NB'
    )
    return node_devices

In [ ]:
def get_devices_on_circuits_sql(
    circuit_nbs
):
    r"""
    """
    #--------------------------------------------------
    assert(Utilities.are_all_list_elements_of_type(lst = circuit_nbs, typ = str))
    #-------------------------
    sql_select = SQLSelect(
        field_descs               = ['GIS_CIRCUIT_NB', 'UPSTREAM_LOCATION', 'LOCATION_NB'], 
        global_table_alias_prefix = None
    )
    #-------------------------
    sql_from = SQLFrom(
        schema_name = 'COMMON', 
        table_name  = 'COM_DGIS_CIRCUIT_DEVICES', 
        alias       = None
    )
    #-------------------------
    sql_where = SQLWhere()
    sql_where = SQLWhere.add_where_statement_equality_or_in(
        sql_where           = sql_where, 
        field_desc          = 'GIS_CIRCUIT_NB', 
        value               = circuit_nbs, 
        needs_quotes        = True, 
        table_alias_prefix  = None, 
        idx                 = None, 
        run_check           = False
    )
    #-------------------------
    sql = SQLQuery(
        sql_select  = sql_select, 
        sql_from    = sql_from, 
        sql_where   = sql_where, 
        sql_groupby = None, 
        alias       = None
    )
    #-------------------------
    sql_stmnt = sql.get_sql_statement()
    return sql_stmnt

In [ ]:
def get_devices_on_circuits(
    circuit_nbs , 
    batch_size  = 1000, 
    conn_eddsp  = None
):
    r"""
    """
    #--------------------------------------------------
    if conn_eddsp is None:
        conn_eddsp = Utilities.get_eddsp_oracle_connection()
    #-------------------------
    build_sql_function = get_devices_on_circuits_sql
    build_sql_function_kwargs = dict(
        circuit_nbs    = circuit_nbs, 
        field_to_split = 'circuit_nbs', 
        batch_size     = batch_size
    )
    #-----
    return_df = GenAn.build_df_general(
        conn_db                        = conn_eddsp, 
        build_sql_function             = build_sql_function, 
        build_sql_function_kwargs      = build_sql_function_kwargs, 
        cols_and_types_to_convert_dict = None, 
        to_numeric_errors              = 'coerce', 
        save_args                      = False, 
        return_sql                     = False, 
        read_sql_args                  = None
    )
    #-------------------------
    return return_df

In [ ]:
def get_prems_from_iso_devs_sql(
    devc_location_nbs
):
    r"""
    """
    #--------------------------------------------------
    assert(Utilities.are_all_list_elements_of_type(lst = devc_location_nbs, typ = str))
    #-------------------------
    sql_select = SQLSelect(
        field_descs               = ['PREM_NB', 'DEVC_LOCATION_NB', 'DEVC_TYPE'], 
        global_table_alias_prefix = None
    )
    #-------------------------
    sql_from = SQLFrom(
        schema_name = 'COMMON', 
        table_name  = 'COM_DGIS_ISOLATING_DEVICES', 
        alias       = None
    )
    #-------------------------
    sql_where = SQLWhere()
    sql_where = SQLWhere.add_where_statement_equality_or_in(
        sql_where           = sql_where, 
        field_desc          = 'DEVC_LOCATION_NB', 
        value               = devc_location_nbs, 
        needs_quotes        = True, 
        table_alias_prefix  = None, 
        idx                 = None, 
        run_check           = False
    )
    #-------------------------
    sql = SQLQuery(
        sql_select  = sql_select, 
        sql_from    = sql_from, 
        sql_where   = sql_where, 
        sql_groupby = None, 
        alias       = None
    )
    #-------------------------
    sql_stmnt = sql.get_sql_statement()
    return sql_stmnt

In [ ]:
def get_prems_from_iso_devs(
    devc_location_nbs , 
    batch_size        = 1000, 
    conn_eddsp        = None
):
    r"""
    """
    #--------------------------------------------------
    if conn_eddsp is None:
        conn_eddsp = Utilities.get_eddsp_oracle_connection()
    #-------------------------
    build_sql_function = get_prems_from_iso_devs_sql
    build_sql_function_kwargs = dict(
        devc_location_nbs = devc_location_nbs, 
        field_to_split    = 'devc_location_nbs', 
        batch_size        = batch_size
    )
    #-----
    return_df = GenAn.build_df_general(
        conn_db                        = conn_eddsp, 
        build_sql_function             = build_sql_function, 
        build_sql_function_kwargs      = build_sql_function_kwargs, 
        cols_and_types_to_convert_dict = None, 
        to_numeric_errors              = 'coerce', 
        save_args                      = False, 
        return_sql                     = False, 
        read_sql_args                  = None
    )
    #-------------------------
    return return_df

In [ ]:
# loc_cir_pairs = long_unmatched_df.reset_index()[['location_nb', 'circuit_nb']].value_counts().index
loc_cir_pairs = alarms_df_full[['location_nb', 'circuit_nb']].value_counts().index

In [ ]:
circ_devs_df = get_devices_on_circuits(circuit_nbs=loc_cir_pairs.get_level_values(1).unique().tolist())
circ_devs_df

In [ ]:
start = time.time()

dwnstrm_from_loc_dfs_0 = []
for location_nb_i, circuit_nb_i in loc_cir_pairs:
    # if (location_nb_i, circuit_nb_i) in [
    #     ('1847269743447','0000306'), 
    #     ('1841347383934','0024802'), 
    #     ('2449886594797','0036476'), 
    #     ('2115086485214','0022902'), 
    #     ('1870483754109','0000921'), 
    #     ('1874214586790','0014906'), 
    #     ('1874595732982','0004204'), 
    #     ('1873118753935','0000921'), 
    #     ('1886581718781','0001008'), 
    #     ('2082512295668','0013805'), 
    #     ('1878155763450','0005501'), 
    #     ('2114945484254','0022902'), 
    #     ('1877130765898','0005501'), 
    #     ('1833355387591','0024802'), 
    # ]:
    #     continue
    print(f"{circuit_nb_i},{location_nb_i}")
    circ_dev_df_i = circ_devs_df[circ_devs_df['GIS_CIRCUIT_NB']==circuit_nb_i]
    dwnstrm_dvcs_i = get_downstream_from_node_in_circ_dev_df_i_v0(
        circ_dev_df_i      = circ_dev_df_i, 
        start_node         = location_nb_i, 
        upstrm_col         = 'UPSTREAM_LOCATION', 
        loc_nb_col         = 'LOCATION_NB', 
        include_start_node = True
    )
    if dwnstrm_dvcs_i is not None:
        dwnstrm_from_loc_df_i = pd.DataFrame(data={
            'location_nb' : location_nb_i, 
            'circuit_nb'  : circuit_nb_i, 
            'dwnstrm_dvc' : [dwnstrm_dvcs_i]
        })
        dwnstrm_from_loc_dfs_0.append(dwnstrm_from_loc_df_i)
        print(f"SUCCESS! {circuit_nb_i},{location_nb_i}\n\n")
#-------------------------
dwnstrm_from_loc_df_0 = pd.concat(dwnstrm_from_loc_dfs_0, axis=0).set_index(['location_nb', 'circuit_nb'])

print(time.time()-start)

In [ ]:
# ('1847269743447', '0000306')

In [ ]:
start = time.time()

dwnstrm_from_loc_dfs = []
for location_nb_i, circuit_nb_i in loc_cir_pairs:
    print(f"{circuit_nb_i},{location_nb_i}")
    circ_dev_df_i = circ_devs_df[circ_devs_df['GIS_CIRCUIT_NB']==circuit_nb_i]
    dwnstrm_dvcs_i = get_downstream_from_node_in_circ_dev_df_i(
        circ_dev_df_i      = circ_dev_df_i, 
        start_node         = location_nb_i, 
        upstrm_col         = 'UPSTREAM_LOCATION', 
        loc_nb_col         = 'LOCATION_NB', 
        include_start_node = True
    )
    # if dwnstrm_dvcs_i is not None:
    dwnstrm_from_loc_df_i = pd.DataFrame(data={
        'location_nb' : location_nb_i, 
        'circuit_nb'  : circuit_nb_i, 
        'dwnstrm_dvc' : [dwnstrm_dvcs_i]
    })
    dwnstrm_from_loc_dfs.append(dwnstrm_from_loc_df_i)
    print(f"SUCCESS! {circuit_nb_i},{location_nb_i}\n\n")
#-------------------------
dwnstrm_from_loc_df = pd.concat(dwnstrm_from_loc_dfs, axis=0).set_index(['location_nb', 'circuit_nb'])

print(time.time()-start)

In [ ]:
assert(0)

In [ ]:
print(dwnstrm_from_loc_df_0.shape)
print(dwnstrm_from_loc_df.shape)

In [ ]:
ovlp_idx = list(set(dwnstrm_from_loc_df_0.index).intersection(set(dwnstrm_from_loc_df.index)))

In [ ]:
df0 = dwnstrm_from_loc_df_0.loc[ovlp_idx].reset_index().copy()
df1 = dwnstrm_from_loc_df.loc[ovlp_idx].reset_index().copy()

df0['dwnstrm_dvc'] = df0['dwnstrm_dvc'].apply(sorted)
df1['dwnstrm_dvc'] = df1['dwnstrm_dvc'].apply(sorted)

In [ ]:
df0.equals(df1)

In [ ]:
dwnstrm_from_loc_df.loc[list(set(dwnstrm_from_loc_df.index).difference(set(dwnstrm_from_loc_df_0.index)))]['dwnstrm_dvc']

In [ ]:
list(set(dwnstrm_from_loc_df_0.index).difference(set(dwnstrm_from_loc_df.index)))

In [ ]:
start = time.time()

dwnstrm_from_loc_dfs_2 = []
for location_nb_i, circuit_nb_i in loc_cir_pairs:
    if (location_nb_i, circuit_nb_i) in [
        ('1847269743447','0000306'), 
        ('1841347383934','0024802'), 
        ('2449886594797','0036476'), 
        ('2115086485214','0022902'), 
        ('1870483754109','0000921'), 
        ('1874214586790','0014906'), 
        ('1874595732982','0004204'), 
        ('1873118753935','0000921'), 
        ('1886581718781','0001008'), 
        ('2082512295668','0013805'), 
        ('1878155763450','0005501'), 
        ('2114945484254','0022902'), 
        ('1877130765898','0005501'), 
        ('1833355387591','0024802'), 
    ]:
        continue
    print(f"{circuit_nb_i},{location_nb_i}")
    dwnstrm_dvcs_i = get_downstream_from_node_in_circuit_bilj(
        circuit_str = circuit_nb_i, 
        start_node  = location_nb_i
    )
    if dwnstrm_dvcs_i is not None:
        dwnstrm_from_loc_df_i = pd.DataFrame(data={
            'location_nb' : location_nb_i, 
            'circuit_nb'  : circuit_nb_i, 
            'dwnstrm_dvc' : [dwnstrm_dvcs_i]
        })
        dwnstrm_from_loc_dfs_2.append(dwnstrm_from_loc_df_i)
        print(f"SUCCESS! {circuit_nb_i},{location_nb_i}\n\n")
#-------------------------
dwnstrm_from_loc_df_2 = pd.concat(dwnstrm_from_loc_dfs_2, axis=0).set_index(['location_nb', 'circuit_nb'])

print(time.time()-start)

In [ ]:
# Using Biljana's method results in duplicates as well, which must be removed
dwnstrm_from_loc_df_2['dwnstrm_dvc'] = dwnstrm_from_loc_df_2['dwnstrm_dvc'].apply(np.unique)

In [ ]:
dwnstrm_from_loc_df.loc[dwnstrm_from_loc_df_2.index]

In [ ]:
df1 = dwnstrm_from_loc_df.loc[dwnstrm_from_loc_df_2.index].sort_index().reset_index().copy()
df2 = dwnstrm_from_loc_df_2.sort_index().reset_index().copy()

In [ ]:
df1['dwnstrm_dvc'] = df1['dwnstrm_dvc'].apply(sorted)
df2['dwnstrm_dvc'] = df2['dwnstrm_dvc'].apply(sorted)

In [ ]:
df1.equals(df2)

In [ ]:
df1

In [ ]:
dwnstrm_from_loc_df.shape

In [ ]:
dwnstrm_from_loc_df_0.shape

In [ ]:
dwnstrm_from_loc_df_2.shape

In [ ]:
dwnstrm_from_loc_df_0

In [ ]:
assert(0)

In [ ]:
# def build_tree_bilj(
#     upstream, 
#     node
# ):
#     tree = defaultdict(list)
#     for u, n in zip(upstream, node):
#         tree[u].append(n)
#     return tree

In [ ]:
circ_dev_df_i.shape

In [ ]:
upstrm_col    = 'UPSTREAM_LOCATION'
loc_nb_col    = 'LOCATION_NB'
tree = build_tree_bilj(
    upstream = circ_dev_df_i[upstrm_col].tolist(), 
    node     = circ_dev_df_i[loc_nb_col].tolist()
)

In [ ]:
circ_dev_df_i.shape

In [ ]:
len(tree)

In [ ]:
PN_to_isoDev_df = get_prems_from_iso_devs(
    devc_location_nbs = dwnstrm_from_loc_df['dwnstrm_dvc'].explode().unique().tolist(), 
    batch_size        = 1000, 
    conn_eddsp        = None
)

In [ ]:
PN_to_isoDev_df